Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

    SELECT name FROM `Facilities` WHERE membercost <> 0;


Q2: How many facilities do not charge a fee to members?

    SELECT count(name) FROM `Facilities` WHERE membercost = 0;

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

    SELECT facid, name, membercost, monthlymaintenance 
    FROM `Facilities` 
    WHERE membercost < 0.2*monthlymaintenance 
    AND membercost <> 0;

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

    SELECT * FROM `Facilities` WHERE facid IN (1,5); 

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

    SELECT name, monthlymaintenance,
	CASE 
		WHEN monthlymaintenance > 100 THEN 'expensive' 
		ELSE 'cheap' 
	END value
    FROM Facilities;

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

    SELECT firstname, surname
    FROM `Members`
    WHERE joindate = (
    SELECT MAX( joindate )
    FROM Members )

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

    SELECT Facilities.name, Members.firstname, Members.surname 
    FROM Bookings
    INNER JOIN Members
    ON Bookings.memid = Members.memid
    INNER JOIN Facilities
    ON Bookings.facid = Facilities.facid
    WHERE Bookings.facid IN (0,1)
    GROUP BY Members.firstname
    ORDER BY Members.surname;

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

    SELECT Facilities.name, CONCAT(Members.firstname, ' ', Members.surname) AS name,
    CASE 
        WHEN Bookings.memid = 0 THEN Facilities.guestcost * Bookings.slots
        ELSE Facilities.membercost * Bookings.slots
    END as cost
    FROM Bookings
    INNER JOIN Members
    ON Bookings.memid = Members.memid
    INNER JOIN Facilities
    ON Bookings.facid = Facilities.facid
    WHERE Bookings.starttime LIKE '2012-09-14%'
    HAVING cost > 30
    ORDER BY cost DESC;

Q9: This time, produce the same result as in Q8, but using a subquery.

    SELECT Facilities.name, CONCAT(Members.firstname, ' ', Members.surname) AS name,
    CASE 
        WHEN subquery1.memid = 0 THEN Facilities.guestcost * subquery1.slots
        ELSE Facilities.membercost * subquery1.slots
    END as cost
    FROM (SELECT * FROM Bookings WHERE Bookings.starttime LIKE '2012-09-14%') subquery1
    INNER JOIN Members
    ON subquery1.memid = Members.memid
    INNER JOIN Facilities
    ON subquery1.facid = Facilities.facid
    HAVING cost > 30
    ORDER BY cost DESC;

In [7]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!


import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT Facilities.name, 
            SUM(CASE WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost
                ELSE Bookings.slots * Facilities.membercost
                END) as Revenue
        FROM Bookings
        INNER JOIN Facilities
        USING (facid)
        GROUP BY Facilities.name
        HAVING Revenue < 1000
        ORDER BY Revenue DESC;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [14]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT a.surname || ', ' || a.firstname Member,
           NULLIF(b.surname || ', ' ||  b.firstname,'GUEST, GUEST') Recommender
        FROM Members a
        LEFT JOIN Members b
        ON a.recommendedby = b.memid
        ORDER BY Member;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Farrell, David', None)
('Farrell, Jemima', None)
('GUEST, GUEST', None)
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rownam, Tim', None)
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Darren', None)
('Smith, Darren', None)
('Smith, Jack', 'Smith, Darren')
('Smith, Tracy', None)
('Stibbons, Ponder', 'Tracy, Burton')
('Tracy, Burton', None)
('Tuppe

In [15]:
# Q12: Find the facilities with their usage by member, but not guests

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
      SELECT Facilities.name, SUM(Bookings.slots) Activity
        FROM Facilities
        INNER JOIN Bookings
        USING (facid)
        WHERE Bookings.memid <> 0
        GROUP BY Facilities.name
        ORDER BY Activity DESC;
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 1086)
('Tennis Court 1', 957)
('Massage Room 1', 884)
('Tennis Court 2', 882)
('Snooker Table', 860)
('Pool Table', 856)
('Table Tennis', 794)
('Squash Court', 418)
('Massage Room 2', 54)


In [18]:
# Q13: Find the facilities usage by month, but not guests

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
      SELECT Facilities.name, SUM(Bookings.slots) Activity, 
        CASE WHEN SUBSTR(Bookings.starttime, 6, 2) = '01' THEN 'JANUARY'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '02' THEN 'FEBRUARY'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '03' THEN 'MARCH'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '04' THEN 'APRIL'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '05' THEN 'MAY'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '06' THEN 'JUNE'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '07' THEN 'JULY'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '08' THEN 'AUGUST'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '09' THEN 'SEPTEMBER'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '10' THEN 'OCTOBER'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '11' THEN 'NOVEMBER'
             WHEN SUBSTR(Bookings.starttime, 6, 2) = '12' THEN 'DECEMBER'
             ELSE 'Error'
        END AS Month	
    FROM Facilities
    INNER JOIN Bookings
    USING (facid)
    WHERE Bookings.memid <> 0
    GROUP BY Facilities.name, Month
    ORDER BY Facilities.name, SUBSTR(Bookings.starttime, 6, 2);
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 165, 'JULY')
('Badminton Court', 414, 'AUGUST')
('Badminton Court', 507, 'SEPTEMBER')
('Massage Room 1', 166, 'JULY')
('Massage Room 1', 316, 'AUGUST')
('Massage Room 1', 402, 'SEPTEMBER')
('Massage Room 2', 8, 'JULY')
('Massage Room 2', 18, 'AUGUST')
('Massage Room 2', 28, 'SEPTEMBER')
('Pool Table', 110, 'JULY')
('Pool Table', 303, 'AUGUST')
('Pool Table', 443, 'SEPTEMBER')
('Snooker Table', 140, 'JULY')
('Snooker Table', 316, 'AUGUST')
('Snooker Table', 404, 'SEPTEMBER')
('Squash Court', 50, 'JULY')
('Squash Court', 184, 'AUGUST')
('Squash Court', 184, 'SEPTEMBER')
('Table Tennis', 98, 'JULY')
('Table Tennis', 296, 'AUGUST')
('Table Tennis', 400, 'SEPTEMBER')
('Tennis Court 1', 201, 'JULY')
('Tennis Court 1', 339, 'AUGUST')
('Tennis Court 1', 417, 'SEPTEMBER')
('Tennis Court 2', 123, 'JULY')
('Tennis Court 2', 345, 'AUGUST')
('Tennis Court 2', 414, 'SEPTEMBER')
